In [1]:
from   fundopt.fundopt import FundTargetRetMinCVaROptimiser, FundTargetRetMinVarianceOptimiser
from   fundopt.fundtsloader import load_funds
import pandas   as pd
import numpy    as np
import datetime as dt
import logging

In [2]:
startDate = dt.date(2014, 3, 23)
endDate   = dt.date(2020, 3, 24)

fundList  = np.genfromtxt( 'refData/AvailableFundList.txt', dtype = str ).tolist()
fundList.remove('003254')
fundList.remove('003255')

returns = load_funds( fundList, startDate, endDate, holdingPeriod=20 )


In [10]:
returns.to_pickle( '2014-03-23_2020-03-24_20.pkl' )

In [ ]:
startDate = dt.date(2019, 9, 23)
endDate   = dt.date(2020, 3, 24)

fundList  = np.genfromtxt( 'refData/AvailableFundList.txt', dtype = str ).tolist()
fundList.remove('003254')
fundList.remove('003255')

to_map = partial( _load_data, start = startDate, end = endDate )
pool = mp.Pool(8)
res = pool.map( to_map, fundList[:100] )
pool.close()
pool.join()
a# res = _load_data( '161903', startDate, endDate )

In [1]:
import tushare as ts

In [3]:
df = ts.get_nav_open()

[Getting data:]#list index out of range


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'